# Contexte


La maladie d'Alzheimer est une maladie neurodégénérative progressive qui affecte la mémoire, la pensée et le comportement. Un diagnostic précoce et précis est crucial pour une prise en charge optimale et pour la recherche de traitements, étant donné l'absence actuelle de remède. Distinguer les stades de la maladie ou l'absence de démence est essentiel pour la recherche et l'amélioration de la qualité de vie des patients.

Ce projet utilise un dataset synthétique riche, conçu pour explorer les facteurs liés à l'Alzheimer et développer des modèles prédictifs. Il contient une variété de caractéristiques : démographiques, comportementales, cliniques et liées aux habitudes de vie. Cela inclut l'âge, les antécédents familiaux, le niveau d'éducation, le tabagisme, l'activité physique, l'alimentation, le sommeil, la tension artérielle, le cholestérol, ainsi que les scores d'évaluation cognitive (MMSE, ADL) et divers symptômes.

Chaque entrée dans le dataset représente un patient, décrit par ces caractéristiques. La variable cible est le diagnostic de la maladie d'Alzheimer, qui sera traitée comme une tâche de classification (présence ou absence de la maladie, ou ses différents stades).

L'objectif de ce projet est de développer des modèles de prédiction précis capables d'utiliser les caractéristiques fournies pour anticiper avec justesse le diagnostic de la maladie d'Alzheimer. Ces modèles pourraient potentiellement aider les professionnels de la santé à prendre des décisions éclairées, en identifiant rapidement les individus à risque ou ceux nécessitant une évaluation médicale approfondie. En conséquence, cela pourrait contribuer à améliorer la détection précoce de la maladie et la qualité de vie des patients affectés.

# Problématique

Face à la complexité et à la nature progressive de la maladie d'Alzheimer, un diagnostic précoce et précis reste un défi majeur. Les données cliniques et comportementales des patients étant multiples et hétérogènes, comment peut-on développer un modèle de Deep Learning capable d'extraire efficacement les informations pertinentes de ces nombreuses caractéristiques pour prédire de manière fiable le diagnostic d'Alzheimer, en s'appuyant sur une représentation de données plus compacte et optimisée ?

# Analyse descriptive

In [3]:
# Importation des librairies
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn as sk 

from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm 
from sklearn import metrics 

In [6]:
# Chargement du dataset
data = pd.read_csv("alzheimers_disease_data.csv", header=0)

In [7]:
# Aperçu du dataset (affiche les 3 premières lignes)
data.head(5)

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


# Description des attributs


Pour notre projet de Deep Learning, nous avons identifié 22 variables jugées les plus pertinentes parmi le dataset "Alzheimer's Disease Dataset" de Kaggle. Ces variables serviront de base à l'entraînement de notre auto-encodeur, qui aura pour objectif de générer une représentation latente de 12 dimensions pour notre Réseau de Neurones Artificiels (ANN).

Voici la liste des attributs sélectionnées, regroupées par catégories pour une meilleure compréhension :

## A. Variables Cognitives et Fonctionnelles :

1.  **MMSE**: Score du Mini-Mental State Examination. Il s'agit d'un indicateur clé de la fonction cognitive globale, dont un score bas est fortement corrélé à une déficience.
2.  **FunctionalAssessment**: Score d'évaluation fonctionnelle. Cette variable mesure la capacité du patient à réaliser les activités de la vie quotidienne, un aspect crucial pour évaluer l'autonomie et la sévérité de la démence.
3.  **ADL (Activities of Daily Living)**: Score des Activités de la Vie Quotidienne. Similaire à l'évaluation fonctionnelle, il quantifie l'impact de la maladie sur les tâches de routine.
4.  **MemoryComplaints**: Indique la présence de plaintes de mémoire. C'est un symptôme précoce et très fréquent, souvent le premier signe d'un trouble cognitif.
5.  **DifficultyCompletingTasks**: Indique la présence de difficultés à accomplir des tâches. Ce symptôme est un signe direct d'une atteinte cognitive affectant les fonctions exécutives.
6.  **Forgetfulness**: Indique la présence d'oublis. C'est un symptôme cardinal de la maladie d'Alzheimer, souvent progressif.

## B. Symptômes Comportementaux et Psychiatriques :

7.  **BehavioralProblems**: Indique la présence de problèmes comportementaux. Ces troubles peuvent accompagner la démence et impacter significativement la vie du patient et de son entourage.
8.  **Confusion**: Indique la présence de confusion. Un état de confusion est un signe aigu ou chronique d'altération de la cognition.
9.  **Disorientation**: Indique la présence de désorientation. La désorientation spatiale ou temporelle est un symptôme courant des démences.
10. **PersonalityChanges**: Indique la présence de changements de personnalité. Ces altérations peuvent être marquantes et sont souvent liées à l'évolution de la maladie.
11. **Agitation**: Indique la présence d'agitation. L'agitation est un symptôme comportemental fréquent chez les personnes atteintes de démence.
12. **Depression**: Indique la présence de dépression. La dépression peut être un facteur de risque pour la démence ou un symptôme précoce/comorbidité de la maladie.

## C. Facteurs de Risque Médicaux et Antécédents :

13. **Age**: L'âge du patient. C'est le facteur de risque le plus puissant pour le développement de la maladie d'Alzheimer.
14. **FamilyHistoryAlzheimers**: Indique la présence d'antécédents familiaux d'Alzheimer. Cela suggère une prédisposition génétique ou environnementale.
15. **Hypertension**: Indique la présence d'hypertension. L'hypertension artérielle est un facteur de risque vasculaire connu pour les démences.
16. **Diabetes**: Indique la présence de diabète. Le diabète est un facteur de risque métabolique et vasculaire pour la démence.
17. **CardiovascularDisease**: Indique la présence de maladies cardiovasculaires. La santé cardiovasculaire est étroitement liée à la santé cérébrale.
18. **HeadInjury**: Indique des antécédents de traumatisme crânien. Les traumatismes crâniens sont des facteurs de risque connus pour le développement de maladies neurodégénératives.

## D. Mesures Cliniques et Mode de Vie :

19. **SystolicBP**: Pression artérielle systolique. Un indicateur important de la santé vasculaire générale.
20. **CholesterolTotal**: Niveaux de cholestérol total. Les niveaux de cholestérol sont liés à la santé cardiovasculaire et, potentiellement, à la santé cérébrale.
21. **Smoking**: Statut tabagique. Le tabagisme est un facteur de risque modifiable pour de nombreuses maladies, y compris les démences.
22. **SleepQuality**: Qualité du sommeil. Une mauvaise qualité de sommeil est de plus en plus reconnue comme un facteur de risque ou un marqueur précoce de l'Alzheimer, en raison de son impact sur la clearance des protéines toxiques dans le cerveau.

## Information sur le dataset 

In [8]:
# Prévisualisation des type de données de notre dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PatientID                  2149 non-null   int64  
 1   Age                        2149 non-null   int64  
 2   Gender                     2149 non-null   int64  
 3   Ethnicity                  2149 non-null   int64  
 4   EducationLevel             2149 non-null   int64  
 5   BMI                        2149 non-null   float64
 6   Smoking                    2149 non-null   int64  
 7   AlcoholConsumption         2149 non-null   float64
 8   PhysicalActivity           2149 non-null   float64
 9   DietQuality                2149 non-null   float64
 10  SleepQuality               2149 non-null   float64
 11  FamilyHistoryAlzheimers    2149 non-null   int64  
 12  CardiovascularDisease      2149 non-null   int64  
 13  Diabetes                   2149 non-null   int64